In [16]:
import os
# Uncomment the following if you have more than 6 GB GPU mem
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import tensorflow as tf
import soundfile as sf
from heuristics import core, tsms
from fb_qualities import FBQualities, FBQuality
from IPython.display import Audio

In [8]:
fbq = FBQualities()
fbq.build()

In [17]:
frame_step = 64
sr = 16000


def get_measure(audio_file, note, measure: FBQuality):
    audio, sr = sf.read(audio_file)
    audio = tf.cast(audio, dtype=tf.float32)
    audio = tf.reshape(audio, shape=(1, -1))

    f0_estimate = tsms.core.midi_to_f0_estimate(
        note, audio.shape[1], frame_step)

    refined_f0_estimate, _, _ = tsms.core.refine_f0(
        audio, f0_estimate, sr, frame_step)

    h_freq, h_mag, h_phase = tsms.core.iterative_harmonic_analysis(
        signals=audio,
        f0_estimate=refined_f0_estimate,
        sample_rate=sr,
        frame_step=frame_step,
        corr_periods_list=[8.0] * 4,
        freq_smoothing_window=21)

    harmonic = tsms.core.harmonic_synthesis(
        h_freq, h_mag, h_phase, sr, frame_step)
    harmonic = harmonic[:, :audio.shape[1]]

    residual = audio - harmonic

    freq_range = measure.frequencies
    m = core.frequency_band_measure(h_freq, h_mag, h_phase,
                                    residual, sample_rate=sr,
                                    frame_step=frame_step,
                                    f_min=freq_range[0],
                                    f_max=freq_range[1])

    return m.numpy()

In [18]:
file_path = "samples/guitar_electronic_003-060-050_harmonic.wav"

for n in fbq.names:
    m = get_measure(file_path,
            note=60, measure=eval(f"fbq.{n}"))
    print(n, m)

thin 0.7788049
hollow 0.31654775
dark 0.0066234805
warm 0.8419335
full 0.33246902
bright 0.010374503
muddy 0.8419335
boxy 0.19122434
honky 0.16796811
harsh 0.18840425
tinny 0.059573267
sibilance 0.07185956


In [19]:
Audio(file_path, rate=sr)